In [121]:
import re
import glob
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib
from copy import deepcopy
import copy
from scipy import stats
from scipy.stats import pearsonr
from matplotlib.pyplot import figure
import shutil
import xlwings as xw
import statistics

%matplotlib inline
mpl.use("Agg")

import warnings
warnings.filterwarnings('ignore')

# 1. Data Preparation

## Patients' Final Food Items

In [115]:
wb = xw.Book(r'D:\diets_5Y\20251204finalFooditems.xlsx')
sheet = wb.sheets("7-days").used_range.value
df_patAllFoods = pd.DataFrame(sheet)

In [116]:
df_patAllFoods.columns = df_patAllFoods.iloc[0]

In [117]:
df_patAllFoods = df_patAllFoods[1:]

In [ ]:
df_patAllFoods  # "r_number", "FoodID", "FoodName", Weight [float]

In [119]:
df_patAllFoods = df_patAllFoods[["r_number", "FoodName", "Weight"]]

## Recipes

In [7]:
wb = xw.Book(r'D:\diets_5Y\Baseline Diet Data 20231222.xlsx')
sheet1 = wb.sheets("Recipe").used_range.value
sheet2 = wb.sheets("Fat").used_range.value
sheet3 = wb.sheets("Sugar").used_range.value
sheet4 = wb.sheets("Condiments").used_range.value
df_recipe = pd.DataFrame(sheet1)
df_fat = pd.DataFrame(sheet2)
df_sugar = pd.DataFrame(sheet3)
df_condiments = pd.DataFrame(sheet4)

In [8]:
df_recipe.columns = df_recipe.iloc[0]
df_recipe = df_recipe[1:]

In [ ]:
df_recipe  # "Food item", "Ingredients", Adjusted Weight [NoneType], Percentage [float]

## Fat

In [10]:
df_fat.columns = df_fat.iloc[0]
df_fat = df_fat[1:]

In [ ]:
df_fat  # "Food item", "Constituent", Percentage [float]

## Sugar

In [12]:
df_sugar.columns = df_sugar.iloc[0]
df_sugar = df_sugar[1:]

In [ ]:
df_sugar  # "Food item", "Constituent", Percentage [float]

## Condiments

In [14]:
df_condiments.columns = df_condiments.iloc[0]
df_condiments = df_condiments[1:]

In [ ]:
df_condiments  # "Food item", "Component", Percentage [float]

## HEI 300 Codes

In [16]:
wb = xw.Book(r'D:\diets_5Y\HEI_narrow_codes.xlsx')
sheet300 = wb.sheets("300 codes").used_range.value
df_300 = pd.DataFrame(sheet300)

In [17]:
df_300.columns = df_300.iloc[0]
df_300 = df_300[1:]

In [ ]:
df_300  # 39 "cat." 39 "cat. code"

## Unit Conversion, g to cup

In [19]:
wb = xw.Book(r'D:\diets_5Y\unit conversions.xlsx')
sheetg2cup = wb.sheets("g to cup").used_range.value
sheetg2oz = wb.sheets("g to oz").used_range.value
sheetCatCodes = wb.sheets("HEI categorized codes").used_range.value
df_g2cup = pd.DataFrame(sheetg2cup)
df_g2oz = pd.DataFrame(sheetg2oz)
df_CatCodes = pd.DataFrame(sheetCatCodes)

In [20]:
df_g2cup.columns = df_g2cup.iloc[0]
df_g2cup = df_g2cup[1:]

In [ ]:
df_g2cup  # "Code", one cup equivalent in g [float], "Food", 
        # "Appendix from 2013-2041", "Outside source", "Remarks"

# Unit Conversion, g to oz

In [22]:
df_g2oz.columns = df_g2oz.iloc[0]
df_g2oz = df_g2oz[1:]

In [ ]:
df_g2oz  # "Code", one cup equivalent in oz [float], "Food", 
        # "Appendix from 2013-2041", "Outside source", "Remarks"

## HEI Categorized Codes

In [24]:
df_CatCodes.columns = df_CatCodes.iloc[0]
df_CatCodes = df_CatCodes[1:]

In [ ]:
df_CatCodes  # 13 cat.

## Patients' Energy Intake from Diets

In [26]:
wb = xw.Book(r'D:\diets_5Y\20251204_diets.xlsx')
sheetE = wb.sheets("7-days").used_range.value
df_energy = pd.DataFrame(sheetE)

In [27]:
df_energy.columns = df_energy.iloc[0]
df_energy = df_energy[1:]

In [ ]:
df_energy  # "r_number", EnergyDF_kJ [float]... mind the units

# 2. from diet weight to atomic ingredients weight with HEI codes

In [365]:
def searchAtomic(df, pat, foodName, name, i, row):
    for j in range(len(df)):
        item = df.iloc[j]["Food item"].replace('"', '').replace("not further defined", "").replace(" ", '')
        temp = [pat]
        if item == foodName:
            #print(df.iloc[j][name])
            temp.append(df.iloc[j][name])
            per = df.iloc[j]["Percentage"]
            if type(per) == float:
                temp.append(df_patAllFoods.iloc[i]["Weight"] *(df.iloc[j]["Percentage"]/100))
            else:
                temp.append(0)
            row.loc[len(row)] = temp
    return row

In [366]:
def searchFood(pat, i, row):
    foodName = df_patAllFoods.iloc[i]["FoodName"].replace('"', '').replace("not further defined", "").replace(" ", '')
    list_df = [df_recipe, df_fat, df_sugar, df_condiments]
    list_dfName = ["Recipe", "Fat", "Sugar", "Condiments"]
    list_colName = ["Ingredients", "Constituent", "Component", "Component"]
    for x in range(len(list_df)):
        outdf_patFoods = searchAtomic(list_df[x], pat, foodName, list_colName[x], i, row)
        if len(outdf_patFoods) > 1:
            outdf_patFoods["TerminatedPoint"] = list_dfName[x]
            outdf_patFoods.at[0, "TerminatedPoint"] = list_dfName[x] + "^_^"
            break
    if len(outdf_patFoods) == 1:
        outdf_patFoods["TerminatedPoint"] = "@w@"
    return outdf_patFoods

In [367]:
row = copy.deepcopy(df_patAllFoods)[0:0+1].reset_index().drop(columns=["index"])
outdf_patAllFoods = searchFood(df_patAllFoods.iloc[0]["r_number"], 0, row)
outdf_patAllFoods["TerminatedPoint"] = "Recipe"
outdf_patAllFoods.at[0,"TerminatedPoint"] = "Recipe" + "^_^"

In [368]:
outdf_patAllFoods

,r_number,FoodName,Weight,TerminatedPoint
0,R0001,Congee with preserved egg and minced pork,165.9143,Recipe^_^
1,R0001,Preserved Egg,9.954858,Recipe
2,R0001,Rice Porridge,124.435725,Recipe
3,R0001,Pork,31.523717,Recipe


In [ ]:
for i in range(1,len(df_patAllFoods)):
    row = copy.deepcopy(df_patAllFoods)[i:i+1].reset_index().drop(columns=["index"])
    course = searchFood(df_patAllFoods.iloc[i]["r_number"], i, row)
    outdf_patAllFoods = pd.concat([outdf_patAllFoods, course])
    print("Course #"+ str(i))

Course #1
Course #2
Course #3
Course #4
Course #5
Course #6
Course #7
Course #8
Course #9
Course #10
Course #11
Course #12
Course #13
Course #14
Course #15
Course #16
Course #17
Course #18
Course #19
Course #20
Course #21
Course #22
Course #23
Course #24
Course #25
Course #26
Course #27
Course #28
Course #29
Course #30
Course #31
Course #32
Course #33
Course #34
Course #35
Course #36
Course #37
Course #38
Course #39
Course #40
Course #41
Course #42
Course #43
Course #44
Course #45
Course #46
Course #47
Course #48
Course #49
Course #50
Course #51
Course #52
Course #53
Course #54
Course #55
Course #56
Course #57
Course #58
Course #59
Course #60
Course #61
Course #62
Course #63
Course #64
Course #65
Course #66
Course #67
Course #68
Course #69
Course #70
Course #71
Course #72
Course #73
Course #74
Course #75
Course #76
Course #77
Course #78
Course #79
Course #80
Course #81
Course #82
Course #83
Course #84
Course #85
Course #86
Course #87
Course #88
Course #89
Course #90
Course #91
Course #

Course #693
Course #694
Course #695
Course #696
Course #697
Course #698
Course #699
Course #700
Course #701
Course #702
Course #703
Course #704
Course #705
Course #706
Course #707
Course #708
Course #709
Course #710
Course #711
Course #712
Course #713
Course #714
Course #715
Course #716
Course #717
Course #718
Course #719
Course #720
Course #721
Course #722
Course #723
Course #724
Course #725
Course #726
Course #727
Course #728
Course #729
Course #730
Course #731
Course #732
Course #733
Course #734
Course #735
Course #736
Course #737
Course #738
Course #739
Course #740
Course #741
Course #742
Course #743
Course #744
Course #745
Course #746
Course #747
Course #748
Course #749
Course #750
Course #751
Course #752
Course #753
Course #754
Course #755
Course #756
Course #757
Course #758
Course #759
Course #760
Course #761
Course #762
Course #763
Course #764
Course #765
Course #766
Course #767
Course #768
Course #769
Course #770
Course #771
Course #772
Course #773
Course #774
Course #775
Cour

Course #1347
Course #1348
Course #1349
Course #1350
Course #1351
Course #1352
Course #1353
Course #1354
Course #1355
Course #1356
Course #1357
Course #1358
Course #1359
Course #1360
Course #1361
Course #1362
Course #1363
Course #1364
Course #1365
Course #1366
Course #1367
Course #1368
Course #1369
Course #1370
Course #1371
Course #1372
Course #1373
Course #1374
Course #1375
Course #1376
Course #1377
Course #1378
Course #1379
Course #1380
Course #1381
Course #1382
Course #1383
Course #1384
Course #1385
Course #1386
Course #1387
Course #1388
Course #1389
Course #1390
Course #1391
Course #1392
Course #1393
Course #1394
Course #1395
Course #1396
Course #1397
Course #1398
Course #1399
Course #1400
Course #1401
Course #1402
Course #1403
Course #1404
Course #1405
Course #1406
Course #1407
Course #1408
Course #1409
Course #1410
Course #1411
Course #1412
Course #1413
Course #1414
Course #1415
Course #1416
Course #1417
Course #1418
Course #1419
Course #1420
Course #1421
Course #1422
Course #1423

Course #1978
Course #1979
Course #1980
Course #1981
Course #1982
Course #1983
Course #1984
Course #1985
Course #1986
Course #1987
Course #1988
Course #1989
Course #1990
Course #1991
Course #1992
Course #1993
Course #1994
Course #1995
Course #1996
Course #1997
Course #1998
Course #1999
Course #2000
Course #2001
Course #2002
Course #2003
Course #2004
Course #2005
Course #2006
Course #2007
Course #2008
Course #2009
Course #2010
Course #2011
Course #2012
Course #2013
Course #2014
Course #2015
Course #2016
Course #2017
Course #2018
Course #2019
Course #2020
Course #2021
Course #2022
Course #2023
Course #2024
Course #2025
Course #2026
Course #2027
Course #2028
Course #2029
Course #2030
Course #2031
Course #2032
Course #2033
Course #2034
Course #2035
Course #2036
Course #2037
Course #2038
Course #2039
Course #2040
Course #2041
Course #2042
Course #2043
Course #2044
Course #2045
Course #2046
Course #2047
Course #2048
Course #2049
Course #2050
Course #2051
Course #2052
Course #2053
Course #2054

Course #2609
Course #2610
Course #2611
Course #2612
Course #2613
Course #2614
Course #2615
Course #2616
Course #2617
Course #2618
Course #2619
Course #2620
Course #2621
Course #2622
Course #2623
Course #2624
Course #2625
Course #2626
Course #2627
Course #2628
Course #2629
Course #2630
Course #2631
Course #2632
Course #2633
Course #2634
Course #2635
Course #2636
Course #2637
Course #2638
Course #2639
Course #2640
Course #2641
Course #2642
Course #2643
Course #2644
Course #2645
Course #2646
Course #2647
Course #2648
Course #2649
Course #2650
Course #2651
Course #2652
Course #2653
Course #2654
Course #2655
Course #2656
Course #2657
Course #2658
Course #2659
Course #2660
Course #2661
Course #2662
Course #2663
Course #2664
Course #2665
Course #2666
Course #2667
Course #2668
Course #2669
Course #2670
Course #2671
Course #2672
Course #2673
Course #2674
Course #2675
Course #2676
Course #2677
Course #2678
Course #2679
Course #2680
Course #2681
Course #2682
Course #2683
Course #2684
Course #2685

...

Course #3871
Course #3872
Course #3873
Course #3874
Course #3875
Course #3876
Course #3877
Course #3878
Course #3879
Course #3880
Course #3881
Course #3882
Course #3883
Course #3884
Course #3885
Course #3886
Course #3887
Course #3888
Course #3889
Course #3890
Course #3891
Course #3892
Course #3893
Course #3894
Course #3895
Course #3896
Course #3897
Course #3898
Course #3899
Course #3900
Course #3901
Course #3902
Course #3903
Course #3904
Course #3905
Course #3906
Course #3907
Course #3908
Course #3909
Course #3910
Course #3911
Course #3912
Course #3913
Course #3914
Course #3915
Course #3916
Course #3917
Course #3918
Course #3919
Course #3920
Course #3921
Course #3922
Course #3923
Course #3924
Course #3925
Course #3926
Course #3927
Course #3928
Course #3929
Course #3930
Course #3931
Course #3932
Course #3933
Course #3934
Course #3935
Course #3936
Course #3937
Course #3938
Course #3939
Course #3940
Course #3941
Course #3942
Course #3943
Course #3944
Course #3945
Course #3946
Course #3947

Course #4502
Course #4503
Course #4504
Course #4505
Course #4506
Course #4507
Course #4508
Course #4509
Course #4510
Course #4511
Course #4512
Course #4513
Course #4514
Course #4515
Course #4516
Course #4517
Course #4518
Course #4519
Course #4520
Course #4521
Course #4522
Course #4523
Course #4524
Course #4525
Course #4526
Course #4527
Course #4528
Course #4529
Course #4530
Course #4531
Course #4532
Course #4533
Course #4534
Course #4535
Course #4536
Course #4537
Course #4538
Course #4539
Course #4540
Course #4541
Course #4542
Course #4543
Course #4544
Course #4545
Course #4546
Course #4547
Course #4548
Course #4549
Course #4550
Course #4551
Course #4552
Course #4553
Course #4554
Course #4555
Course #4556
Course #4557
Course #4558
Course #4559
Course #4560
Course #4561
Course #4562
Course #4563
Course #4564
Course #4565
Course #4566
Course #4567
Course #4568
Course #4569
Course #4570
Course #4571
Course #4572
Course #4573
Course #4574
Course #4575
Course #4576
Course #4577
Course #4578

Course #5133
Course #5134
Course #5135
Course #5136
Course #5137
Course #5138
Course #5139
Course #5140
Course #5141
Course #5142
Course #5143
Course #5144
Course #5145
Course #5146
Course #5147
Course #5148
Course #5149
Course #5150
Course #5151
Course #5152
Course #5153
Course #5154
Course #5155
Course #5156
Course #5157
Course #5158
Course #5159
Course #5160
Course #5161
Course #5162
Course #5163
Course #5164
Course #5165
Course #5166
Course #5167
Course #5168
Course #5169
Course #5170
Course #5171
Course #5172
Course #5173
Course #5174
Course #5175
Course #5176
Course #5177
Course #5178
Course #5179
Course #5180
Course #5181
Course #5182
Course #5183
Course #5184
Course #5185
Course #5186
Course #5187
Course #5188
Course #5189
Course #5190
Course #5191
Course #5192
Course #5193
Course #5194
Course #5195
Course #5196
Course #5197
Course #5198
Course #5199
Course #5200
Course #5201
Course #5202
Course #5203
Course #5204
Course #5205
Course #5206
Course #5207
Course #5208
Course #5209

Course #5764
Course #5765
Course #5766
Course #5767
Course #5768
Course #5769
Course #5770
Course #5771
Course #5772
Course #5773
Course #5774
Course #5775
Course #5776
Course #5777
Course #5778
Course #5779
Course #5780
Course #5781
Course #5782
Course #5783
Course #5784
Course #5785
Course #5786
Course #5787
Course #5788
Course #5789
Course #5790
Course #5791
Course #5792
Course #5793
Course #5794
Course #5795
Course #5796
Course #5797
Course #5798
Course #5799
Course #5800
Course #5801
Course #5802
Course #5803
Course #5804
Course #5805
Course #5806
Course #5807
Course #5808
Course #5809
Course #5810
Course #5811
Course #5812
Course #5813
Course #5814
Course #5815
Course #5816
Course #5817
Course #5818
Course #5819
Course #5820
Course #5821
Course #5822
Course #5823
Course #5824
Course #5825
Course #5826
Course #5827
Course #5828
Course #5829
Course #5830
Course #5831
Course #5832
Course #5833
Course #5834
Course #5835
Course #5836
Course #5837
Course #5838
Course #5839
Course #5840

Course #6395
Course #6396
Course #6397
Course #6398
Course #6399
Course #6400
Course #6401
Course #6402
Course #6403
Course #6404
Course #6405
Course #6406
Course #6407
Course #6408
Course #6409
Course #6410
Course #6411
Course #6412
Course #6413
Course #6414
Course #6415
Course #6416
Course #6417
Course #6418
Course #6419
Course #6420
Course #6421
Course #6422
Course #6423
Course #6424
Course #6425
Course #6426
Course #6427
Course #6428
Course #6429
Course #6430
Course #6431
Course #6432
Course #6433
Course #6434
Course #6435
Course #6436
Course #6437
Course #6438
Course #6439
Course #6440
Course #6441
Course #6442
Course #6443
Course #6444
Course #6445
Course #6446
Course #6447
Course #6448
Course #6449
Course #6450
Course #6451
Course #6452
Course #6453
Course #6454
Course #6455
Course #6456
Course #6457
Course #6458
Course #6459
Course #6460
Course #6461
Course #6462
Course #6463
Course #6464
Course #6465
Course #6466
Course #6467
Course #6468
Course #6469
Course #6470
Course #6471

total patients: 366


In [86]:
type(list(df_energy["EnergyDF_kJ"])[0])

float

In [ ]:
outdf_patAllFoods.to_csv("D:\diets_5Y\out\outputTable1_.csv")